# Entity Flow Examples


## Objects Definitions

In [ ]:
# Entity Flow Models.

import simpy
import numpy as np
from typing import Callable
import matplotlib.pyplot as plt
import pandas as pd


class Entity():
    """Abstrac object that flows throw the system"""
    def __init__(self, id, create_time=None):
        self.id = id
        self.created_time = create_time
        self.deleted_time = None


class Queu():
    """Stores entities with specific get and put policy."""
    def __init__(self, env:simpy.Environment,name, capacity=float('inf')):
        self.env = env
        self.name = name
        self.store = simpy.Store(env,capacity=capacity)

    def put(self, entity:Entity):         
        yield self.store.put(entity)
        print(f"{self.env.now} - entity{entity.id} put in {self.name}")


    def get(self):
        e = yield self.store.get()
        print(f"{self.env.now} - entity{e.id} get from {self.name}")
        return e



class Source():
    """Creates entities with an interval arrival tiem"""

    def __init__(self, env:simpy.Environment, name:str, interarrival_time:Callable, max_quantity=float("inf")):
        self.env = env
        self.name = name
        self.interarrival_time = interarrival_time
        self.max_quantity = max_quantity
        self.total_created = 0

        self._output = None

        self.action = env.process(self.run())

    @property
    def output(self):
        return self._output

    @output.setter
    def output(self, output):
        if callable(getattr(output, "put", None)):
            self._output = output
        else:
            raise Exception("Output must have a put() method")


    def run(self):
        while True:
            yield self.env.timeout(self.interarrival_time)
            e = Entity(id=self.total_created,create_time=self.env.now)
            print(f"{self.env.now} - entity{e.id} generated in {self.name}")
            self.total_created += 1
            try:
                yield self.env.process(self._output.put(e))
            except:
                Exception("An output must be assigned")
            



class Sink():
    """Delete entities and records it."""
    def __init__(self, env:simpy.Environment, name:str):
        self.env = env
        self.name = name
        self.sincked_entities = Queu(env, name=f"{name}_queu")
        self.total_sinked = 0
        
    def put(self, entity:Entity):
        print(f"{self.env.now} - entity{entity.id} sinked in {self.name}")
        yield self.env.process(self.sincked_entities.put(entity))
        entity.deleted_time = self.env.now
        self.total_sinked += 1
        


class Server():
    """Delay entities for a given time"""
    def __init__(self, env:simpy.Environment, name:str, server_time=10):
        self.env = env
        self.name = name
        self.server_time = server_time
        self.queu = Queu(env, name=f"{name}_queu")

        self.total_processed = 0
        self._output = None

        self.action = env.process(self.run())

    def run(self):
        while True:
            e = yield self.env.process(self.queu.get())
            yield self.env.timeout(self.server_time)
            print(f"{self.env.now} - entity{e.id} processed in {self.name}")
            try:
                yield self.env.process(self._output.put(e))
                self.total_processed += 1
            except:
                Exception("An output must be assigned")

    def put(self, entity):
        yield self.env.process(self.queu.put(entity))
        # self.queu.put(entity)

    @property
    def output(self):
        return self._output

    @output.setter
    def output(self, output):
        if callable(getattr(output, "put", None)):
            self._output = output
        else:
            raise Exception("Output must have a put() method")
        


class Brancher():
    """Split entities in out branches by a condition"""
    def __init__(self, env:simpy.Environment,name:str, probs=[0.3,0.7]):
        self.env = env
        self.name = name
        self.probs = probs
        self.ranges = [sum(probs[0:n+1]) for n in range(len(probs))]
        self._output = [None for i in range(len(probs))]


    @property
    def output(self):
        return self._output

    def set_output(self, idx, output):
        if callable(getattr(output, "put", None)):
            self._output[idx] = output
        else:
            raise Exception("Output must have a put() method")


    def put(self, entity):
        rand = np.random.random()
        for i in range(len(self.probs)):
            if rand <= self.ranges[i]:
                print(f"{self.env.now} - entity{entity.id} assigned to output {i} in {self.name}")
                yield self.env.process(self._output[i].put(entity))



## 01-One Server System

In [ ]:
# one server
env = simpy.Environment()

source = Source(env=env, name="source", interarrival_time=5)
server = Server(env=env, name='server', server_time=10)
sink = Sink(env=env, name='sink')

# build conections
source.output = server
server.output = sink
env.run(until=100)

## 02-Two sequential server System

In [ ]:
# two servers
env = simpy.Environment()

source = Source(env=env, name="source", interarrival_time=5)
server1 = Server(env=env, name='server1', server_time=10)
server2 = Server(env=env, name='server2', server_time=10)
sink = Sink(env=env, name='sink')

# build conections
source.output = server1
server1.output = server2
server2.output = sink
env.run(until=100)

In [ ]:
print(server1.total_processed)
print(server2.total_processed)
print(sink.total_sinked)

## 03-Branched System

In [ ]:
# three servers with a branch.

env = simpy.Environment()

source = Source(env=env, name="source", interarrival_time=5)

server1 = Server(env=env, name='server1', server_time=10)
server2 = Server(env=env, name='server2', server_time=5)
server3 = Server(env=env, name='server3', server_time=5)

brancher = Brancher(env=env, name='brancher', probs=[0.3, 0.7])

sink1 = Sink(env=env, name='sink1')
sink2 = Sink(env=env, name='sink2')

# build conections
source.output = server1
server1.output = brancher
brancher.set_output(0, server2)
brancher.set_output(1, server3)
server2.output = sink1
server3.output = sink2

env.run(until=100)

In [ ]:
print("Server 1: ",server1.total_processed)
print("Server 2: ",server2.total_processed)
print("Server 3: ",server3.total_processed)
print("Sink 1: ", sink1.total_sinked)
print("Sink 2: ",sink2.total_sinked)

## 04-Branched and joined system

In [ ]:
# four servers with a branch and a final server.

env = simpy.Environment()

source = Source(env=env, name="source", interarrival_time=5)

server1 = Server(env=env, name='server1', server_time=10)
server2 = Server(env=env, name='server2', server_time=5)
server3 = Server(env=env, name='server3', server_time=5)
server4 = Server(env=env, name='server4', server_time=10)

brancher = Brancher(env=env, name='brancher', probs=[0.3, 0.7])

sink = Sink(env=env, name='sink1')

# build conections
source.output = server1
server1.output = brancher
brancher.set_output(0, server2)
brancher.set_output(1, server3)
server2.output = server4
server3.output = server4
server4.output = sink
env.run(until=100)

In [ ]:
print("Server 1: ",server1.total_processed)
print("Server 2: ",server2.total_processed)
print("Server 3: ",server3.total_processed)
print("Server 4: ",server4.total_processed)
print("Sink: ", sink.total_sinked)